# Camera calibration and some transformations operations

In [6]:
import numpy as np
import os
import os.path as osp
import cv2 as cv

In [16]:
def read_cam_file(filename):
    """
    Load camera file e.g., 00000000_cam.txt
    """
    with open(filename) as f:
        lines = [line.rstrip() for line in f.readlines()]
    # extrinsics: line [1,5), 4x4 matrix
    extrinsics = np.fromstring(' '.join(lines[1:5]), dtype=np.float32, sep=' ')
    extrinsics = extrinsics.reshape((4, 4))
    # intrinsics: line [7-10), 3x3 matrix
    intrinsics = np.fromstring(' '.join(lines[7:10]), dtype=np.float32, sep=' ')
    intrinsics = intrinsics.reshape((3, 3))
    intrinsics_ = np.float32(np.diag([1, 1, 1, 1]))
    intrinsics_[:3, :3] = intrinsics
    P = intrinsics_ @ extrinsics
    print(intrinsics_)
    print(extrinsics)

    return P, intrinsics_, extrinsics

In [17]:
file_path = "/data1/zhanghm/Code/NeRF/VolRecon/DTU_TEST/cameras/00000023_cam.txt"
P, intrin, extrin = read_cam_file(file_path)
print(P, P.shape)

[[2.89233e+03 0.00000e+00 8.23203e+02 0.00000e+00]
 [0.00000e+00 2.88318e+03 6.19070e+02 0.00000e+00]
 [0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00]
 [0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00]]
[[ 2.64733e-01 -8.47168e-01  4.60677e-01 -3.09358e+02]
 [ 3.92519e-01  5.31021e-01  7.50963e-01 -4.68705e+02]
 [-8.80821e-01 -1.79800e-02  4.73108e-01  3.56303e+02]
 [ 0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00]]
[[ 4.0600708e+01 -2.4650908e+03  1.7218938e+03 -6.0145575e+05]
 [ 5.8641302e+02  1.5198982e+03  2.4580483e+03 -1.1307842e+06]
 [-8.8082099e-01 -1.7980000e-02  4.7310799e-01  3.5630301e+02]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]] (4, 4)


In [21]:
out = cv.decomposeProjectionMatrix(P[:3, :4])
print(type(out), len(out))

K = out[0]
R = out[1]
t = out[2]

K = K / K[2, 2]
intrinsics = np.eye(4)
intrinsics[:3, :3] = K

pose = np.eye(4, dtype=np.float32)
pose[:3, :3] = R.transpose()
pose[:3, 3] = (t[:3] / t[3])[:, 0]  # cam to world

print(intrinsics)
print(pose)

diff_intri = intrinsics - intrin
diff_pose = np.linalg.inv(pose) - extrin
print(diff_intri)
print(diff_pose, diff_pose.min(), diff_pose.max())

<class 'tuple'> 7
[[2.89233081e+03 1.72135202e-04 8.23202087e+02 0.00000000e+00]
 [0.00000000e+00 2.88317969e+03 6.19069641e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 2.6473260e-01  3.9251891e-01 -8.8082093e-01  5.7971173e+02]
 [-8.4716785e-01  5.3102106e-01 -1.7980000e-02 -6.7794447e+00]
 [ 4.6067697e-01  7.5096309e-01  4.7310796e-01  3.2592432e+02]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[ 0.00073242  0.00017214 -0.00091553  0.        ]
 [ 0.         -0.00024414 -0.00036621  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[[-3.5762787e-07  1.7881393e-07  0.0000000e+00  2.4414062e-04]
 [-8.9406967e-08  5.9604645e-08  1.1920929e-07  6.1035156e-05]
 [ 0.0000000e+00  9.3132257e-09  0.0000000e+00 -6.1035156e-05]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]] -6.1035156e-05 0.0